# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

In [65]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

vroom_url = 'http://vroom:3000/optimization/'

vehicleProfile = "driving-car"

# DATI posizioni
geoA = json.load(open('../data/mezzolombardo.geojson'))['features']   #4 points
geoB = json.load(open('../data/mezzocorona.geojson'))['features']     #6 points
geoC = json.load(open('../data/piana_rotaliana.geojson'))['features'] #6 points distanti

pointsA = [ p['geometry']['coordinates'] for p in geoA ]
pointsB = [ p['geometry']['coordinates'] for p in geoB ]
pointsC = [ p['geometry']['coordinates'] for p in geoC ]

drt_example = json.load(open('../data/drt_2_veicoli_2_corse.json')) #from vroom shipments example

for v in drt_example['vehicles']:
    v['profile'] = vehicleProfile

center = [
    11.1126,
    46.2083
]
zoom = 14

point_start =  [11.10249,46.21579] #posizione iniziale veicoli
point_start2 = [11.11258,46.20834]
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

def vehicle2Map(v):
    idv = str(v['id'])    
    loc = v['start']
    cap = str(v['capacity'][0])

    try:
        times = v['time_window']
    except:
        times = []
    
    mText = "VEICOLO %s <br> capacity %s" % (idv, cap)
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = 'red'#COLORS[idv]
    
    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = 'green'#COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "CORSA %s <br>" % s['pickup']['id']
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Partenza orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    #mText += "Quantita %s <br>" % s['amount'][0]
    
    mark = folium.Marker(revcc(loc1), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color=color))
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "CORSA %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Arrivo orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    #mText += "Quantita %s <br>" % s['amount'][0]
    
    mark = folium.Marker(revcc(loc2), tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color=color))
    mark.add_to(Map)

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple',
    '4': 'green'
}

HOURS1 = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]

HOURS2 = [
    [
        hour2sec('14:00'),
        hour2sec('14:05')
    ],
    [
        hour2sec('15:00'),
        hour2sec('15:05')
    ],
    [
        hour2sec('16:00'),
        hour2sec('16:05')
    ],
    [
        hour2sec('17:00'),
        hour2sec('17:05')
    ],
    [
        hour2sec('18:00'),
        hour2sec('18:05')
    ]
]
# VEICOLI

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [1],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        hour2sec('07:00'),
        hour2sec('13:00')
    ],
    #TODO
    #     'breaks': [              #se ci sono orari precisi di pause e sanitizzazione del veicolo
    #         {
    #           "id": 1,
    #           #"description": "pausa sanitizzazione", #descrizione pausa
    #           "service": 300,   #durata pausa 5 minuti
    #           "time_windows": [ #an array of time_window objects describing valid slots for break start
    #             [
    #               hour2sec('13:05'),
    #               hour2sec('13:55')
    #             ],
    #           ]
    #         }
    #       ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = drt_example['vehicles'][1]['start']

vehicle = {
    'id': vid,
    'capacity': [1],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        hour2sec('14:00'),
        hour2sec('19:00')
    ]
}
vehicles.append(vehicle)

#DEFINITION OF JOBS

job_locs = pointsA + pointsB

jobs = []
for jid, cc in enumerate(job_locs):
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna/ritiro(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    #orario casuali
    timeWindow = random.choice(HOURS1)
    
    # shifting casuale del tempo per ogni job
    #s = random.randrange(-3000,3000)
    #timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    job = {
        'id': jid,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna o aggiunta del tempo DI PULIZIA mezzo
        'time_windows': [ timeWindow ]
    }
    jobs.append(job)

request = {
     'vehicles': vehicles,
     'jobs': [],#jobs,
     'shipments': []
}

ships = []
for ship in drt_example['shipments']:
    #ship['pickup']['time_windows']= [[    #pickup == PARTENZA in orario 
    #ship['pickup']['time_windows']= [[]]
    
    shiptype = random.choice(['pickup','delivery'])
    
    ship[ shiptype ]['time_windows']= [    #delivery == ARRIVO in orario
        random.choice(HOURS1)
        #[hour2sec('11:00'),hour2sec('11:05')]
    ]
    ship[ shiptype ]['location'] = pointsC.pop()
    
    ships.append(ship)

request['shipments'] = ships

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

for j in request['jobs']:
    job2Map(j)

for s in request['shipments']:
    #print(json.dumps(s, indent=4))
    ship2Map(s)
    
Map

In [70]:
#request

In [66]:
request['geometry'] = True       #RITORNA GEOMETRIA DELLA SOLUZIONE

resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
solution

{'code': 0,
 'summary': {'cost': 1578,
  'unassigned': 0,
  'delivery': [2],
  'amount': [2],
  'pickup': [2],
  'service': 0,
  'duration': 1578,
  'waiting_time': 2167,
  'priority': 0,
  'distance': 20082,
  'computing_times': {'loading': 12, 'solving': 0, 'routing': 11}},
 'unassigned': [],
 'routes': [{'vehicle': 1,
   'cost': 1578,
   'delivery': [2],
   'amount': [2],
   'pickup': [2],
   'service': 0,
   'duration': 1578,
   'waiting_time': 2167,
   'priority': 0,
   'distance': 20082,
   'steps': [{'type': 'start',
     'location': [11.10649, 46.21979],
     'load': [0],
     'arrival': 1617793249,
     'duration': 0,
     'distance': 0},
    {'type': 'pickup',
     'location': [11.08717918395996, 46.214169599073806],
     'id': 3,
     'service': 0,
     'waiting_time': 0,
     'job': 3,
     'load': [1],
     'arrival': 1617793500,
     'duration': 251,
     'distance': 2325},
    {'type': 'delivery',
     'location': [11.15713119506836, 46.22349332052325],
     'id': 3,
   

In [82]:
#MAP RESULT
Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

for v in request['vehicles']:
    vehicle2Map(v)

if len(solution['routes']) > 0:
    for route in solution['routes']:
        
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        
        rcolor = COLORS[str(route['vehicle'])]
        #rcolor = random.choice( list(COLORS.values()) )
        
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)
        
        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)
        
        i=0
        for step in route['steps']:
            i += 1
            try:
                loc = step['location']
            except:
                loc = None
            
            if loc and step['type'] != 'end':
                tit = str(i)+" "+step['type']+"<br> arrivo: %s " % sec2hour(step['arrival'])
                
                mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, direction='bottom', permanent=True, style='background:yellow'), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
                mark.add_to(Map)
            
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))
    
Map

Vehicle1 blue distance: 20km delivered quantity: 2 pickup quantity: 2 , durata: 26min 

Total distance: 20km Not Delivered: 0


In [79]:
#print(json.dumps(solution, indent=2))